# Histogram obrazu. Wyrównywanie histogramu.

## Cel ćwiczenia

- Zapoznanie z pojęciem histogramu obrazu (w odcieniach szarości i kolorze).
- Zapoznanie z metodami modyfikacji histogramu (rozciąganie, wyrównywanie, dopasowywanie).

## Histogram

- Histogramem obrazu nazywamy wykres słupkowy zdefiniowany następującymi zależnościami:<br>
\begin{equation}
h(i) = \sum_{x=0}^{N-1} \sum_{y=0}^{M-1} p(i,(x,y))
\end{equation}<br>
gdzie:<br>
\begin{equation}
p(i) =  \left\{
  \begin{array}{l l}
    1 & \quad \text{gdy} f(x,y) = i\\
    0 & \quad \text{gdy} f(x,y) \ne i
  \end{array} \right.
\end{equation}

- Inaczej mówiąc, histogram zawiera informacje na temat tego ile pikseli o danym poziomie jasności występuje na obrazie (w przypadku obrazu w odcieniach szarości). Określa się to także rozkładem empirycznym cechy.

- Często wykorzystuje się tzw. znormalizowaną postać histogramu  – wszystkie wartości $h(i)$ są dzielone przez liczbę pikseli na obrazie.
Otrzymana w ten sposób wielkość to gęstość prawdopodobieństwa wystąpienia na obrazie pikseli o odcieniu $i$.

- Histogram można zdefiniować również dla obrazów kolorowych.
Otrzymujemy wtedy 3 histogramy – po jednym dla danej składowej: R,G,B (lub HSV, YCbCr, itp.) lub histogram trójwymiarowy.

- Histogram jest bardzo użyteczny w przetwarzaniu i analizie obrazów.
Wykorzystywany jest przy binaryzacji (szerzej na jednym z kolejnych laboratoriów) oraz do oceny jakości (dynamiki, kontrastu) obrazu.
W idealnym przypadku wszystkie poziomy jasności w obrazie powinny być wykorzystane (i to najlepiej w miarę jednolicie)  – obrazowo mówiąc histogram powinien rozciągać się od 0  – 255 (obraz w skali szarości).

- W przypadku gdy  wykorzystujemy jedynie fragment dostępnego zakresu (wąski histogram)  lub histogram nie jest jednolity (występują dominujące grupy pikseli) obraz ma dość słaby kontrast.
Cechę tę można poprawić stosując tzw. rozciąganie albo wyrównywanie histogramu (ang. *histogram equalization*).</div>

## Histogram dla obrazów w odcieniach szarości

1. Zaimportuj potrzebne biblioteki: *OpenCV*, *pyplot* z *matplotlib* i *numpy*.
        import cv2
        from matplotlib import pyplot as plt
        import numpy as np
2. Wczytaj obrazy *lenaX.bmp* w skali szarości. *X* jest numerem wczytywanego obrazu (1 - 4).
        I = cv2.imread('lenaX.bmp', cv2.IMREAD_GRAYSCALE)
3. Oblicz histogram wczytanego obrazu wykorzystując funkcję `cv2.calcHist`.
    - Pierwszym argumentem jest obraz, dla którego obliczony zostanie histogram.
    Należy go przekazać w nawiasie kwadratowym.
    - Drugim argumentem jest numer kanału, dla którego ma zostać obliczony histogram.
    Również powinien być przekazany w nawiasie kwadratowym.
    - Trzeci argument oznacza maskę, czyli obszar, dla którego ma zostać wyznaczony histogram.
    Aby obliczyć dla całego obrazu należy przekazać *None*.
    - Czwartym argumentem jest rozmiar histogramu (liczba przedziałów).
    Argument powinien być w nawiasie kwadratowym. Dla pełnej skali należy przekazać wartość *256*.
    - Ostatnim argumentem jest zakres wartości. Dla obrazów typu *uint8* powinien on wynosić *[0, 256]*.
    - Funkcja zwraca obliczony histogram.
4. Wyświetl wczytane obrazy i ich histogramy w jednym oknie. Użyj `plt.subplot()` w celu stworzenia siatki wykresów.
        figLena, axsLena = plt.subplots(2, 4)
Rozmiar utworzonego okna można zmienić wykorzystując instrukcję (uwaga w calach -  1 cal to 2.54cm):
        figLena.set_size_inches(20, 10)
Przykładowe wyświetlenie obrazu:
        axsLena[0, 0].imshow(I1, 'gray', vmin=0, vmax=256)
        axsLena[0, 0].axis('off')
Przykładowe wyświetlenie histogramu:
        axsLena[1, 0].plot(H1)
        axsLena[1, 0].grid()
5. Przeanalizuj (dokładnie) związek histogramu z jasnością i ostrością obrazu (tu rozumianą jako subiektywne odczucie).

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/'

fileName = 'lena1.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

fileName = 'lena2.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

fileName = 'lena3.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

fileName = 'lena4.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
lena_imgs = [cv2.imread(f'lena{i}.bmp', cv2.IMREAD_GRAYSCALE) for i in range(1, 4 + 1)]
lena_hist = [cv2.calcHist([img], [0], None, [256], [0, 256]) for img in lena_imgs]

for img, hist in zip(lena_imgs, lena_hist):
    fig, axs = plt.subplots(1, 2, figsize=(20, 10), constrained_layout=True)
    axs[0].imshow(img, cmap='gray', vmin=0, vmax=255)
    axs[0].axis('off')
    axs[1].plot(hist)
    axs[1].set_xlim([0, 256])
    axs[1].set_ylim([0, 3000])
    axs[1].set_xlabel('Value')
    axs[1].set_ylabel('Pixels')
    axs[1].grid(True)

plt.show()

## Rozciąganie histogramu

Najprostszą metodą poprawienia jakości obrazu jest tzw. rozciągnięcie histogramu.
Polega na przeskalowaniu wartości pikseli w obrazie tak, aby wykorzystać cały dostępny zakres [0-255] (oczywiście w przypadku obrazów w odcieniach szarości w reprezentacji 8-bitowej).

1. Wczytaj obraz *hist1.bmp* w skali szarości.
Oblicz i wyświetl histogram rozpatrywanego obrazu (na wspólnym rysunku z obrazem).
Zwróć uwagę na ilość widocznych szczegółów.
2. Rozciągnij histogram obrazu. W tym celu można wykorzystać funkcję `cv2.normalize`.
    - Pierwszym argumentem funkcji jest obraz poddawany operacji.
    - Drugim argumentem jest tablica do której zostanie wpisany wynik.
    Należy ją najpierw zainicjalizować.
    Najlepiej zrobić to funkcją `np.zeros`, której pierwszym argumentem jest rozmiar obrazu (`I.shape`), a drugim typ danych (`uint8`).
    Można również przekazać `None`, a wynik przypisać do nowej zmiennej.
    - Trzecim argumentem jest minimalna wartość po normalizacji.
    - Czwartym argumentem jest wartość maksymalna po normalizacji.
    - Ostatnim argumentem jest typ wykorzystanej normy (uogólnienie pojęcia długości wektora).
    Należy wykorzystać normę `cv2.NORM_MINMAX`.
3. Wyświetl obraz oryginalny, po wykonanej operacji oraz ich histogramy.
4. Czy ilość "widocznych" szczegółów uległa zmianie?

In [ ]:
fileName = 'hist1.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

fileName = 'hist2.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

fileName = 'hist3.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

fileName = 'hist4.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
hist_img = cv2.imread(f'hist1.bmp', cv2.IMREAD_GRAYSCALE)
hist_img_normalized = cv2.normalize(hist_img, None, 0, 256, cv2.NORM_MINMAX)
hist_histogram = cv2.calcHist([hist_img], [0], None, [256], [0, 256])
hist_histogram_normalized = cv2.calcHist([hist_img_normalized], [0], None, [256], [0, 256])

# Plot pics and histograms
fig, axs = plt.subplots(1, 4, figsize=(15, 5), constrained_layout=True)
axs[0].imshow(hist_img, cmap='gray', vmin=0, vmax=256)
axs[0].axis('off')
axs[1].imshow(hist_img_normalized, cmap='gray', vmin=0, vmax=256)
axs[1].axis('off')
axs[2].plot(hist_histogram)
axs[2].set_xlabel('Value')
axs[2].set_ylabel('Pixels')
axs[2].grid(True)
axs[3].plot(hist_histogram_normalized)
plt.show()

## Wyrównywanie histogramu

<div style="text-align: justify">
Bardziej zaawansowaną metodą jest tzw. wyrównywanie histogramu (ang. *histogram equalization – HE*).
Idea jest następująca: z punktu widzenia lepszego wykorzystania dostępnych poziomów jasności pożądane jest rozciągnięcie "szczytów" histogramu, a~skompresowanie "dolin" tak, aby taka sama liczba pikseli reprezentowana była przez każdy z dostępnych poziomów jasności (a przynjamniej przybliżona).
Warto zwrócić uwagę, że takie przekształcenie powoduje częściową utratę informacji o szczegółach w obszarach "dolin".
Inaczej mówiąc, dążymy do sytuacji, aby histogram był względnie jednostajny.
Operacją, która pozwala wykonać wyrównywanie histogramu, jest przekształcenie LUT z funkcją przejścia w postaci histogramu skumulowanego danego obrazu.</div><br>

<div style="text-align: justify">
Histogram skumulowany to funkcja obliczona na podstawie histogramu.
Jej pierwszy element to liczba pikseli o odcieniu $0$.
Kolejne wartości to liczba pikseli o odcieniach od $0$ do $n$.</div>

\begin{equation}
C(n) = \sum_{i=0}^{n} h(i)
\end{equation}

<div style="text-align: justify">
Jeżeli histogram jest w postaci znormalizowanej (gęstość rozkładu prawdopodobieństwa) to histogram skumulowany stanowi dystrybuantę rozkładu prawdopodobieństwa.</div><br>

1. Wyznacz histogram skumulowany dla obrazu *hist1.bmp*.
W tym celu wykorzystaj metodę `cumsum` dla histogramu wczytanego obrazu.
Nie przyjmuje ona żadnych argumentów, a zwraca skumulowane wartości tablicy, dla której została użyta.
Histogram należy wyliczyć dla **obrazka wejściowego**, a nie dla wyniku rozciągania.
2. Histogram skumulowany wyświetl razem z histogramem zwykłym na jednym wykresie (nie obok siebie).
Na potrzeby wyświetlenia przeskaluj histogram skumulowany tak, by miał taką samą wartość maksymalną jak zwykły histogram.
W tym celu wykorzystaj metodę `max`.
3. Wyświetlenie kilku linii na jednym wykresie może być zrealizowane w następujący sposób:
        figHistCum, axsHistCum = plt.subplots()

        axsHistCum.plot(HHist)
        axsHistCum.plot(CHistNorm)
        axsHistCum.grid()
4. Teraz zaimplementuj klasyczny algorytm wyrównywania histogramu.
Wykorzystać należy obliczony histogram skumulowany.
Należy go przeskalować w taki sposób aby na jego podstawie zrealizować przekształcenie LUT, czyli do zakresu 0 - 255.

>Uwaga. Opisany algorytm wyrównywania histogramu jest wersją uproszczoną.
>W wersji pełnej należy podczas skalowania tablicy przekodowań LUT pominąć elementy równe *0*.
>
>W tym celu należy wykorzystać funkcje `np.ma.masked_equal` i `np.ma.filled`.
>Pierwsza służy do ukrywania elementów tablicy, natomiast druga zamienia ukryte elementy na podaną wartość.
>W tym przypadku elementem ukrywanym i wpisywaną wartością byłoby *0*.

5. Na kolejnym rysunku wyświetl obrazek po przekształceniu, jego histogram oraz histogram skumulowany.
Co szczególnego można powiedzieć o jego histogramie i histogramie skumulowanym?
6. W bibliotece *OpenCV* dostępna jest funkcja wykonująca wyrównywanie histogramu `cv2.equalizeHist`.
Jej argumentem jest obraz, którego histogram zostanie wyrównany. Zwraca natomiast obraz wynikowy.
Na kolejnym rysunku wyświetl wynik funkcji, jego histogram oraz histogram skumulowany.
7. W wykorzystywanej bibliotece zaimplementowana jest również metoda adaptacyjnego wyrównywania histogramu algorytmem CLAHE (ang. *Contrast Limited Adaptive Histogram Equalization*}.
   Kilka słów wyjaśnienia.
   Wadą poznanej metody HE jest jej "globalność" rozumiana jako nieuwzględnianie lokalnych właściwości obrazu.
   Dlatego też powstała metoda adaptacyjnego wyrównywania histogramu (AHE).
   Jest ona spotykana w dwóch wariantach:
   - dla każdego piksela niezależnie, w pewnym jego otoczeniu, wyznaczany jest histogram i przeprowadzane wyrównywanie.
     Jak nietrudno się domyślić rozwiązanie jest dość kosztowne obliczeniowo.
   - obraz wejściowy dzielony jest na nienachodzące na siebie prostokątne okna.
     W każdym z okien obliczany jest histogram i przeprowadzane jest wyrównywanie.
     W celu eliminacji błędów na granicy okien, stosuje się interpolację.

   Metoda AHE ma jednak tą wadę, że w obszarach jednorodnych wzmacnia szum.
   Dlatego też zaproponowano rozwiązanie CLAHE, które zakłada ograniczenie kontrastu (CL).
   W metodzie definiuje się maksymalną wartość danego przedziału histogramu (próg ograniczenia kontrastu).
   Piksele, które przekraczają próg są następnie równomiernie rozdzielane pomiędzy poszczególne przedziały.
   Bardziej szczegółowy opis obu metod dostępny jest na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization).

8. W celu użycia algorytmu należy wywołać funkcję `cv2.createCLAHE`.
    - Pierwszym parametrem jest próg ograniczenia kontrastu.
    - Drugi parametr definiuje na ile prostokątów zostanie podzielony obraz w rzęch i kolumnach.
    - Zwracany jest zainicjalizowany *smart pointer* do klasy `cv::CLAHE`.
9. Aby wykonać wyrównywanie należy użyć metody `apply`.
Jej argumentem jest obraz wejściowy. Zwracany jest obraz o zmodyfikowanym histogramie.
10. Przetestuj różne parametry algorytmu CLAHE.
11. W kolejnym etapie należy przetestować operacje (rozciąganie, wyrównywanie (HE) i adaptacyjne wyrównywanie CLAHE)  na histogramie dla obrazów rzeczywistych. *hist2.bmp*, *hist3.bmp*, *hist4.jpg*.
W jednym oknie wyświetl: obraz oryginalny, rozciąganie, wyrównywanie HE oraz wyrównywanie CLAHE.

In [ ]:
hist_img = cv2.imread(f'hist1.bmp', cv2.IMREAD_GRAYSCALE)
hist_histogram = cv2.calcHist([hist_img], [0], None, [256], [0, 256])
hist_histogram_cumsum = hist_histogram.cumsum()
hist_histogram_normalized = cv2.normalize(hist_histogram, None, 0, 256, cv2.NORM_MINMAX)
hist_histogram_cumsum_norm = cv2.normalize(hist_histogram_cumsum, None, 0, 256, cv2.NORM_MINMAX)

plt.plot(hist_histogram_normalized)
plt.plot(hist_histogram_cumsum_norm)
plt.grid()
plt.show()

In [ ]:
hist_histogram_cumsum_norm_2 = cv2.normalize(hist_histogram_cumsum, None, 0, 255, cv2.NORM_MINMAX)


circle_img_lut = cv2.LUT(hist_img, hist_histogram_cumsum_norm_2).astype(np.uint8)
circle_hist_lut = cv2.calcHist([circle_img_lut], [0], None, [256], [0, 256])
circle_hist_lut_cumulative = circle_hist_lut.cumsum()

fig, axs = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)
axs[0].imshow(circle_img_lut, cmap='gray', vmin=0, vmax=256)
axs[0].axis('off')
axs[1].plot(circle_hist_lut)
axs[1].set_xlabel('Value')
axs[1].set_ylabel('Pixels')
axs[1].grid(True)
axs[2].plot(circle_hist_lut_cumulative)
plt.show()



In [ ]:
hist_img = cv2.imread(f'hist1.bmp', cv2.IMREAD_GRAYSCALE)
hist_img_eq = cv2.equalizeHist(hist_img)

hist_hist_eq = cv2.calcHist([hist_img_eq], [0], None, [256], [0, 256])
hist_hist_eq_cumsum = hist_hist_eq.cumsum()
fig, axs = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)
axs[0].imshow(hist_img_eq, cmap='gray', vmin=0, vmax=256)
axs[0].axis('off')
axs[1].plot(hist_hist_eq)
axs[1].set_xlabel('Value')
axs[1].set_ylabel('Pixels')
axs[1].grid(True)
axs[2].plot(hist_hist_eq_cumsum)
plt.show()


In [ ]:
def clahe(image):
    _, axs = plt.subplots(len(image), 4, figsize=(15, 5 * len(image)), constrained_layout=True)
    for ax, img in zip(axs, image):
        img_normalized = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        img_he = cv2.equalizeHist(img)
        img_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(img)

        ax[0].imshow(img, cmap='gray', vmin=0, vmax=255)
        ax[0].axis('off')
        ax[0].set_title('Original')
        ax[1].imshow(img_normalized, cmap='gray', vmin=0, vmax=255)
        ax[1].axis('off')
        ax[1].set_title('Normalized')
        ax[2].imshow(img_he, cmap='gray', vmin=0, vmax=255)
        ax[2].axis('off')
        ax[2].set_title('HE')
        ax[3].imshow(img_clahe, cmap='gray', vmin=0, vmax=255)
        ax[3].axis('off')
        ax[3].set_title('CLAHE')

    plt.show()


clahe([cv2.imread(f'hist{i}.bmp', cv2.IMREAD_GRAYSCALE) for i in range(2, 4 + 1)])

## Histogram dla obrazów kolorowych i jego wyrównywanie

1. Wczytaj obraz *lenaRGB.bmp*.
2. Wykonaj konwersję przestrzeni barw z BGR do RGB.
3. Wyświetl wczytany obraz oraz histogram dla każdej składowej przestrzeni barw.
W tym celu można użyć drugiego argumentu wykorzystywanej funkcji (numer kanału).
4. Wykonaj wyrównywanie dla każdej składowej obrazu. Połącz otrzymane składowe w nowy obraz i wyświetl go.
Jaka jest zasadnicza wada takiego podejścia?
5. Przekształć obraz wejściowy do przestrzeni HSV (flaga `cv2.COLOR_BGR2HSV`).
Wyświetl histogramy poszczególnych składowych.
Manipulacji dokonujemy na składowej odpowiadającej za jasność, czyli V.
Wykonaj wyrównywanie histogramu dla tej składowej.
Dokonaj podmiany składowej V i wyświetl rezultat operacji.
Uprzednio przeprowadź konwersję HSV->RGB (flaga `cv2.COLOR_HSV2RGB`).
6. Wykonaj te same operacje dla obrazu *jezioro.jpg*.

In [ ]:
fileName = 'lenaRGB.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

fileName = 'jezioro.jpag'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
lenaRGB, jezioroRGB = cv2.imread('lenaRGB.bmp'), cv2.imread('jezioro.jpg')

lenaRGB = cv2.cvtColor(lenaRGB, cv2.COLOR_BGR2RGB)
jezioroRGB = cv2.cvtColor(jezioroRGB, cv2.COLOR_BGR2RGB)

def RGBHist(img):
    R, G, B = cv2.split(img)
    histograms = [cv2.calcHist([colour], [0], None, [256], [0, 256]) for colour in [R, G, B]]
    _, axs = plt.subplots(1, 4, figsize=(15, 5), constrained_layout=True)
    axs[0].imshow(img)
    axs[0].axis('off')
    axs[1].plot(histograms[0], color='red')
    axs[2].plot(histograms[1], color='green')
    axs[3].plot(histograms[2], color='blue')
    plt.grid()
    plt.show()

RGBHist(lenaRGB)
RGBHist(jezioroRGB)



In [ ]:
def eq4EachRGB(image):
    r, g, b = cv2.split(image)
    rgb_eq = [cv2.equalizeHist(colour) for colour in [r, g, b]]
    img_merged = cv2.merge(rgb_eq)

    _, axs = plt.subplots(1, 5, figsize=(15, 5), constrained_layout=True)
    axs[0].imshow(image)
    axs[0].axis('off')
    for i, c in enumerate(['r', 'g', 'b']):
        axs[i+1].imshow(rgb_eq[i], cmap='gray', vmin=0, vmax=255)
        axs[i+1].axis('off')
    axs[4].imshow(img_merged)
    axs[4].axis('off')

    plt.show()

In [ ]:
eq4EachRGB(lenaRGB)
eq4EachRGB(jezioroRGB)


In [ ]:
lenaRGB, jezioroRGB = cv2.imread('lenaRGB.bmp'), cv2.imread('jezioro.jpg')

In [ ]:
def HistHSV(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img)
    histograms = [cv2.calcHist([brightness], [0], None, [256], [0, 256]) for brightness in [h, s, v]]
    v = cv2.equalizeHist(v)
    hist_eq = [cv2.calcHist([brightness], [0], None, [256], [0, 256]) for brightness in [h, s, v]]
    merged = cv2.merge([h, s, v])
    img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
    img_merged = cv2.cvtColor(merged, cv2.COLOR_HSV2RGB)
    

    _, axs = plt.subplots(2, 4, figsize=(15, 5), constrained_layout=True)
    axs[0][0].imshow(img)
    axs[0][0].axis('off')
    for i in range(len(histograms)):
        axs[0][i+1].plot(histograms[i])
        axs[0][i+1].grid(True)
    axs[1][0].imshow(img_merged)
    axs[1][0].axis('off')
    for i in range(len(hist_eq)):
        axs[1][i+1].plot(hist_eq[i])
        axs[1][i+1].grid(True)

    plt.show()

In [ ]:
HistHSV(lenaRGB)
HistHSV(jezioroRGB)